In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
base_path =  '/content/drive/MyDrive/ML_assignment3/'
debug = 0

In [4]:
!pip install idx2numpy

In [5]:
import math
import random

def cal_loss(probs,N, y) : # probs -> last layer softmax output , N->size of last layer , y->label
    

    tmp = probs[np.arange(N), y]
    qloss = random.uniform(0,1)
    tmp = tmp[tmp != 0]
    loss= 0
    for x in tmp:
      loss += (x*math.log(x))

    loss = (-1)*loss
    

    return qloss

    #  loss = -np.sum(np.log(probs[np.arange(N), y])) / N

In [6]:
import numpy as np


learning_rate = 0.001

essential function for using

In [7]:
def padding( X , pad ): # padding with zero
   X_pad = np.pad(X , ((0,0),(pad,pad),(pad,pad),(0,0)),'constant',constant_values=0)
   return X_pad

#Activation layer: implement an element-wise ReLU.

In [8]:
def relu( input):
    input[input<0] = 0
    return input

def relu_derived(input):
    input[input<0] = 0
    input[input>0] = 1
    return input

#Softmax layer: it will convert final layer projections to normalized probabilities

In [9]:
def  softmax( input):
           input = input - np.max( input , axis=0, keepdims=True)
           return np.exp(input) / np.sum(np.exp(input), axis=0)


#Convolution layer: there will be four (hyper)parameters: the number of output channels,filter dimension, stride, padding.

In [10]:
class Convolution_Layer:
     def __init__(self ,name='conv', num_chanels = 6 , num_prev_chanels =1 , filter_size = 3 , stride = 1 , padding = 0 ,activation =None ):
       self.num_chanels = num_chanels
       self.num_prev_chanels = num_prev_chanels
       self.filter_size = filter_size
       self.stride = stride
       self.padding = padding
       self.W =  np.random.randn( filter_size ,filter_size , num_prev_chanels , num_chanels)
       self.b = np.random.randn( 1,1,1,num_chanels)
       self.last_input = None 
       self.activation = activation
       self.name ='conv'


     def forward(self, input ) : #input must be 4 by 4 matrix which is  output activations of the previous layer, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
        
        self.last_input = input #store for further using 
        (m, n_H_prev, n_W_prev, n_C_prev) = input.shape
        (f_size, f_size, n_C_prev, n_C) = self.W.shape

        # Compute the dimensions of the CONV output volume using the formula given above. Hint: use int() to floor. (≈2 lines)
        n_H = int((n_H_prev - f_size + 2 * self.padding) / self.stride) + 1
        n_W = int((n_W_prev - f_size + 2 * self.padding) / self.stride) + 1
        
        # Initialize the output volume Z with zeros. (≈1 line)
        Z = np.zeros((m, n_H, n_W, n_C))
       
        # Create input_pad by padding input
        input_pad = padding(input, self.padding)
       

        for i in range(m):                               # loop over the batch of training examples
            a_prev_pad = input_pad[i]                               # Select ith training example's padded activation
            for h in range(n_H):
                vert_start = h * self.stride
                vert_end = vert_start + f_size                           # loop over vertical axis of the output volume
                for w in range(n_W):  
                    horiz_start = w * self.stride
                    horiz_end = horiz_start + f_size                     # loop over horizontal axis of the output volume
                    for c in range(n_C):                   # loop over channels (= #filters) of the output volume
                        
                        # Find the corners of the current "slice" (≈4 lines)
                        
                        # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                        a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]
                        
                        # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈1 line)            
                        Z[i, h, w, c] = np.sum (np.multiply(a_slice_prev, self.W[...,c])) + float(self.b[...,c])
                                           
      
        # Making sure your output shape is correct
        assert(Z.shape == (m, n_H, n_W, n_C))  # delete this line of code
        
        if self.activation == 'Relu':
           Z =relu(Z)
        if debug == 1 : 
           print(' Inside convolution layer ->forward ')
           print(Z.shape)
        return Z





      
     def backward(self,dZ):   #last layer data

        
        (m, n_H_prev, n_W_prev, n_C_prev) = self.last_input.shape
        
        # Retrieve dimensions from W's shape
        (f_size, f_size, n_C_prev, n_C) = self.W.shape
        
        
        # Retrieve dimensions from dZ's shape
        (m, n_H, n_W, n_C) = dZ.shape
        
        # Initialize dA_prev, dW, db with the correct shapes
        dA_prev = np.zeros(self.last_input.shape)                           
        dW = np.zeros(self.W.shape)
        db = np.zeros(self.b.shape)

        # Pad A_prev and dA_prev
        A_prev_pad = padding(self.last_input ,self.padding)
        dA_prev_pad = padding(dA_prev , self.padding)
        
        for i in range(m):                       # loop over the training examples
            
            # select ith training example from A_prev_pad and dA_prev_pad
            a_prev_pad = A_prev_pad[i]
            da_prev_pad = dA_prev_pad[i]
            
            for h in range(n_H):                   # loop over vertical axis of the output volume
                vert_start = h * self.stride
                vert_end = vert_start + f_size
                for w in range(n_W):               # loop over horizontal axis of the output volume
                    horiz_start = w * self.stride
                    horiz_end = horiz_start + f_size                      
                    for c in range(n_C):           # loop over the channels of the output volume
                        
                        
                        # Use the corners to define the slice from a_prev_pad
                        a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                        # Update gradients for the window and the filter's parameters using the code formulas given above
                        da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += self.W[:,:,:,c] * dZ[i,h,w,c]
                        dW[:,:,:,c] += a_slice *  dZ[i,h,w,c]
                        db[:,:,:,c] += dZ[i,h,w,c]
                        
            # Set the ith training example's dA_prev to the unpadded da_prev_pad (Hint: use X[pad:-pad, pad:-pad, :])
            #print(i)
            #print(da_prev_pad.shape)
            if self.padding == 0: dA_prev[i,:,:,:] = da_prev_pad
            else : dA_prev[i, :, :, :] = da_prev_pad[self.padding:(-1)*self.padding , self.padding:(-1)*self.padding,:]
        ### END CODE HERE ###
        
        # Making sure your output shape is correct
        assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))

        self.W -= dW * learning_rate
        self.b -= db * learning_rate
        if debug == 1 :
           print(' Inside convolution layer-->backward ')
           print(dA_prev.shape)
        return dA_prev






#Max-pooling layer: there will be two parameters: filter dimension, stride.

In [11]:
from numpy.lib.index_tricks import diag_indices_from
class MaxPooling_Layer:
     def __init__(self ,name='maxPool', filter_size = 2 , stride = 1 ) :

        self.filter_size = filter_size
        self.stride = stride
        self.last_input = None
        self.W = 0
        self.b = 0
        self.name = name

     def forward( self , input ) :
       
        # Retrieve dimensions from the input shape
        self.last_input = input
        (m, n_H_prev, n_W_prev, n_C_prev) = input.shape
        

        # Define the dimensions of the output
        n_H = int(1 + (n_H_prev - self.filter_size) / self.stride)
        n_W = int(1 + (n_W_prev - self.filter_size) / self.stride)
        n_C = n_C_prev
        
        # Initialize output matrix A
        A = np.zeros((m, n_H, n_W, n_C))              
        
        ### START CODE HERE ###
        for i in range(m):                         # loop over the training examples
            for h in range(n_H):                     # loop on the vertical axis of the output volume
                # Find the vertical start and end of the current "slice" (≈2 lines)
                vert_start = self.stride * h 
                vert_end = vert_start + self.filter_size
                
                for w in range(n_W):                 # loop on the horizontal axis of the output volume
                    # Find the vertical start and end of the current "slice" (≈2 lines)
                    horiz_start = self.stride * w
                    horiz_end = horiz_start + self.filter_size
                    
                    for c in range (n_C):            # loop over the channels of the output volume
                        
                        # Use the corners to define the current slice on the ith training example of A_prev, channel c. (≈1 line)
                        a_prev_slice = input[i]
                        A[i, h, w, c] = np.max(a_prev_slice[vert_start:vert_end, horiz_start:horiz_end, c])
                        

        
        # Making sure your output shape is correct
        assert(A.shape == (m, n_H, n_W, n_C))
        if debug == 1 : 
           print(' Inside MaxPooling layer->forward')
           print(A.shape)
        return A



     def backward( self , input):   
       
          
          
          # Retrieve dimensions from A_prev's shape and dA's shape (≈2 lines)
          #print(input.shape)
          m, n_H_prev, n_W_prev, n_C_prev = self.last_input.shape
          m, n_H, n_W, n_C =input.shape
          #print(self.last_input.shape)
          
          # Initialize dA_prev with zeros (≈1 line)
          dA_prev = np.zeros(self.last_input.shape)
          
          for i in range(m):                       # loop over the training examples
              
              # select training example from A_prev (≈1 line)
              a_prev = self.last_input[i]
              da = input[i]
              #print('da :',da.shape)
                        
              for h in range(n_H): 
                  vert_start = self.stride * h 
                  vert_end =  vert_start + self.filter_size                  # loop on the vertical axis
                  for w in range(n_W):               # loop on the horizontal axis
                      horiz_start = self.stride * w
                      horiz_end = horiz_start + self.filter_size
                      for c in range(n_C):           # loop over the channels (depth)
                               
                          # Use the corners and "c" to define the current slice from a_prev (≈1 line)
                          a_prev_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end, c]
                          # Create the mask from a_prev_slice (≈1 line)
                          mask = ( a_prev_slice == (np.max(a_prev_slice)) ) 
                          #print('w:',w , 'h:', h)
                          # Set dA_prev to be dA_prev + (the mask multiplied by the correct entry of dA) (≈1 line)
                         
                          dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += mask * da[h,w,c]
                          
                      
          
          # Making sure your output shape is correct
          assert(dA_prev.shape == self.last_input.shape)
          if debug == 1 : 
            print(' Inside MaxPooling layer->backward ')
            print(dA_prev.shape)
          return dA_prev

#Fully-connected layer: a dense layer. There will be one parameter: output dimension.

In [12]:
class FullyConnected_Layer:

   def __init__( self ,name= 'fullyCon', nodes=100 , num_classes=10) : 
      self.W = np.random.randn(nodes , num_classes) 
      self.b = np.zeros(num_classes)
      self.last_input = None
      self.last_output = None
      self.name = name

   def forward( self , input) :
      
      (b,nh,nw,nc) = input.shape
      self.last_input = input
      #input = input.flatten()
      input_flat = input.reshape(b,1,1,nh*nw*nc)
      if debug == 1 :
        print('inside fcl after flatten input ')
        print(input_flat.shape)
      output = np.dot(input_flat,self.W) + self.b
      #self.last_input = input 
      self.last_output = output
      #---------------------------flatting------------------------------------------------
      (m , n_h , n_w , n_c) = output.shape

      flatting_output = np.zeros(( m , n_h * n_w *n_c))
      for i in range(m):
        flatting_output[i] = softmax(output[i].flatten()) 

      if debug == 1 : 
        print(' Inside Fully connected layer-> forward')
        print(flatting_output.shape)
      return flatting_output

   def backward(self,gradient): #gradient must be (32,10) size 
      N = self.last_input.shape[0]
      new_input = self.last_input.reshape(N , -1)
      dx = np.dot(gradient , self.W.T).reshape(self.last_input.shape)
      dw = np.dot(new_input.T , gradient)
      db = np.sum(gradient.T ,axis = 1)
      self.W -= dw
      self.b -= db
      if debug == 1 :
         print(' Inside Fully connected layer-> backword')
         print(dx.shape)
      return dx






#CNN MODLE
Conv 6 5 1 2 ->
ReLU ->
Pool 2 2 ->
Conv 12 5 1 0 ->
ReLU ->
Pool 2 2 ->
Conv 100 5 1 0 ->
ReLU ->
FC 10 ->
Softmax

In [20]:
import numpy as np
import time


class CNN:
    def __init__(self):
        self.layers = []

    def add_layer(self, layer):
        self.layers.append(layer)

    def build_model(self):
        self.add_layer(Convolution_Layer( num_chanels = 6 , num_prev_chanels =1 , filter_size = 5 , stride = 1 , padding = 2 , activation = 'relu')) # for->minist dataset
        # self.add_layer(Convolution_Layer( num_chanels = 6 , num_prev_chanels =3 , filter_size = 5 , stride = 1 , padding = 2 , activation = 'relu')) # for->ciphar dataset
        self.add_layer(MaxPooling_Layer(filter_size = 2 , stride = 2 ))
        self.add_layer(Convolution_Layer( num_chanels = 12 , num_prev_chanels =6 , filter_size = 5 , stride = 1 , padding = 0 , activation = 'relu'))
        self.add_layer(MaxPooling_Layer(filter_size = 2 , stride = 2 ))
        self.add_layer(Convolution_Layer( num_chanels = 100 , num_prev_chanels =12 , filter_size = 5 , stride = 1 , padding = 0 , activation = 'relu'))
        self.add_layer(FullyConnected_Layer(nodes=100,num_classes=10)) #for ->minis dataset
        # self.add_layer(FullyConnected_Layer(nodes=400,num_classes=10)) #for ->ciphar dataset
    

    def forward(self, input):                # forward propagate
        for layer in self.layers:
            input = layer.forward(input) #input size must be (32,28,28,1)
            # print(layer.name)
            # print(layer.last_input.shape)
        return input

    def backward(self, gradient):                # backward propagate
        for layer in reversed(self.layers):
            # print(layer.name)
            # print(layer.last_input.shape)
            gradient = layer.backward(gradient)

    def train(self, dataset, num_epoch, verbose):
        history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}
        for epoch in range(1, num_epochs + 1):
            print('\n--- Epoch {0} ---'.format(epoch))
            loss, tmp_loss, num_corr = 0, 0, 0
            initial_time = time.time()
            for i in range(0 , len(dataset['train_images']) , 32):
            #for i in range(0 , 1280 , 32):
                if i % 32 == 0:
                    accuracy = (num_corr / (i + 1)) * 100       # compute training accuracy and loss up to iteration i
                    misclassified = ((i - num_corr) / (i+1) ) 
                    #loss = tmp_loss / (i + 1)
                    print('[batch %05d]: Loss %02.3f | Accuracy: %02.3f | Time: %02.2f seconds' %
                          (int(i / 32), misclassified, accuracy, time.time() - initial_time))
                                        
                initial_time = time.time()

                image = dataset['train_images'][i:i+32]
                label = dataset['train_labels'][i:i+32]


                tmp_output = self.forward(image)       # forward propagation

                for eb in range(tmp_output.shape[0]):              
                    if np.argmax(tmp_output[eb]) == label[eb]:                          # update accuracy
                        num_corr += 1

                loss = cal_loss(tmp_output , tmp_output.shape[0] , label)

                dx = tmp_output/tmp_output.shape[0]
                gradient =dx                                      # compute initial gradient
                self.backward(gradient)                      # backward propagation

        print('num_of_corr :',num_corr)
        

    def evaluate(self, X, y,  verbose):
        loss, num_correct = 0, 0
        for i in range(0,len(X),32):
            tmp_output = self.forward(X[i:i+32])              # forward propagation
            

            # compute cross-entropy update loss
            loss = cal_loss(tmp_output , tmp_output.shape[0] , y[i:i+32])
            for eb in range(tmp_output.shape[0]):
                prediction = np.argmax(tmp_output[eb])                              # update accuracy
                if prediction == y[i+eb]:
                    num_correct += 1


        test_size = len(X)
        accuracy = (num_correct / test_size) * 100
        loss = (test_size-num_correct) / test_size
        if verbose:
            print('Test Loss: %02.3f' % loss)
            print('Test Accuracy: %02.3f' % accuracy)
        return loss, accuracy

#dataset_preparing

In [14]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import idx2numpy
import numpy as np
from six.moves import cPickle
import platform
import cv2
sns.set(color_codes=True)


def load_mnist():
    X_train = idx2numpy.convert_from_file(base_path+'MNIST_data/train-images.idx3-ubyte')
    train_labels = idx2numpy.convert_from_file(base_path+'MNIST_data/train-labels.idx1-ubyte')
    X_test = idx2numpy.convert_from_file(base_path+'MNIST_data/t10k-images.idx3-ubyte')
    test_labels = idx2numpy.convert_from_file(base_path+'MNIST_data/t10k-labels.idx1-ubyte')

    train_images = []    
    #print(X_train[0])
    print(np.unique(train_labels))
    print(X_train.shape)
                                                   # reshape train images so that the training set
    for i in range(X_train.shape[0]):                                   # is of shape (60000, 1, 28, 28)
        train_images.append(np.expand_dims(X_train[i], axis=2))
    train_images = np.array(train_images)
    
    #porced = input('input man:')

    print(train_images.shape)
    
    test_images = []                                                    # reshape test images so that the test set
    for i in range(X_test.shape[0]):                                    # is of shape (10000, 1, 28, 28)
        test_images.append(np.expand_dims(X_test[i], axis=2))
    test_images = np.array(test_images)
    

    indices = np.random.permutation(train_images.shape[0])              # permute and split training data in
    training_idx, validation_idx = indices[:55000], indices[55000:]     # training and validation sets
    train_images, validation_images = train_images[training_idx, :], train_images[validation_idx, :]
    train_labels, validation_labels = train_labels[training_idx], train_labels[validation_idx]

    return {
        'train_images': train_images,
        'train_labels': train_labels,
        'validation_images': validation_images,
        'validation_labels': validation_labels,
        'test_images': test_images,
        'test_labels': test_labels
    }



def minmax_normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x - min_val) / (max_val - min_val)
    return x


def preprocess(dataset):
    dataset['train_images'] = np.array([minmax_normalize(x) for x in dataset['train_images']])
    dataset['validation_images'] = np.array([minmax_normalize(x) for x in dataset['validation_images']])
    dataset['test_images'] = np.array([minmax_normalize(x) for x in dataset['test_images']])
    return dataset



def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return cPickle.load(f)
    elif version[0] == '3':
        return cPickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))


def load_CIFAR_batch(filename):
    X_batch = []
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        for i in range(datadict['data'].shape[0]):
            X_batch.append(np.reshape(datadict['data'][i], (3, 32, 32)))
        return np.array(X_batch), np.array(datadict['labels'])


def load_cifar():
    X_train, y_train = [], []
    for batch in range(1, 6):
        X_batch, y_batch = load_CIFAR_batch(os.path.join(base_path+'CIFAR_data', 'data_batch_%d' % batch))
        X_train.append(X_batch)
        y_train.append(y_batch)
    X_train = np.concatenate(X_train)
    y_train = np.concatenate(y_train)
    X_test, y_test = load_CIFAR_batch(os.path.join(base_path+'CIFAR_data', 'test_batch'))

    indices = np.random.permutation(X_train.shape[0])                       # permute and split training data in
    training_idx, validation_idx = indices[:49000], indices[49000:]         # training and validation sets
    X_train, X_val = X_train[training_idx, :], X_train[validation_idx, :]
    y_train, y_val = y_train[training_idx], y_train[validation_idx]
    
    print('different different shape : ')
    X_train = X_train.transpose((0, 2, 3, 1))
    X_test = X_test.transpose((0,2,3,1))
    X_val =  X_val.transpose((0,2,3,1))
    if debug == 1 :
      print('inside load cifer dataset ')
      print(X_train.shape)
      print(X_test.shape)
      print(X_val.shape)
      print(y_train.shape)

    return {
        'train_images': X_train,
        'train_labels': y_train,
        'validation_images': X_val,
        'validation_labels': y_val,
        'test_images': X_test,
        'test_labels': y_test
    }









#Run code

In [21]:
dataset_name = 'mnist'
num_epochs = 1
learning_rate = 0.01
verbose = 1
print('\n--- Loading ' + dataset_name + ' dataset ---')               
dataset = load_mnist() if dataset_name is 'mnist' else load_cifar()
print('\n--- Processing the dataset ---')                               
dataset = preprocess(dataset)
print('\n--- Building the model ---')                                  
model = CNN()
model.build_model()
print('\n--- Training the model ---')                                   
model.train(dataset,num_epochs,verbose)
print('\n--- Testing the model ---')                                    
model.evaluate( dataset['test_images'],dataset['test_labels'],verbose=1)


--- Loading mnist dataset ---
[0 1 2 3 4 5 6 7 8 9]
(60000, 28, 28)
(60000, 28, 28, 1)

--- Processing the dataset ---

--- Building the model ---

--- Training the model ---

--- Epoch 1 ---
[batch 00000]: Loss 0.000 | Accuracy: 0.000 | Time: 0.00 seconds
[batch 00001]: Loss 0.909 | Accuracy: 6.061 | Time: 5.76 seconds
[batch 00002]: Loss 0.908 | Accuracy: 7.692 | Time: 5.87 seconds
[batch 00003]: Loss 0.897 | Accuracy: 9.278 | Time: 6.25 seconds
[batch 00004]: Loss 0.915 | Accuracy: 7.752 | Time: 5.64 seconds
[batch 00005]: Loss 0.913 | Accuracy: 8.075 | Time: 5.87 seconds
[batch 00006]: Loss 0.907 | Accuracy: 8.808 | Time: 5.96 seconds


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


[batch 00007]: Loss 0.916 | Accuracy: 8.000 | Time: 6.01 seconds
[batch 00008]: Loss 0.918 | Accuracy: 7.782 | Time: 5.90 seconds
[batch 00009]: Loss 0.917 | Accuracy: 7.958 | Time: 5.78 seconds
[batch 00010]: Loss 0.910 | Accuracy: 8.723 | Time: 5.75 seconds
[batch 00011]: Loss 0.909 | Accuracy: 8.782 | Time: 5.69 seconds
[batch 00012]: Loss 0.912 | Accuracy: 8.571 | Time: 5.78 seconds
[batch 00013]: Loss 0.914 | Accuracy: 8.393 | Time: 5.83 seconds
[batch 00014]: Loss 0.911 | Accuracy: 8.686 | Time: 5.78 seconds
[batch 00015]: Loss 0.904 | Accuracy: 9.356 | Time: 5.75 seconds
[batch 00016]: Loss 0.904 | Accuracy: 9.357 | Time: 5.87 seconds
[batch 00017]: Loss 0.897 | Accuracy: 10.092 | Time: 6.03 seconds
[batch 00018]: Loss 0.896 | Accuracy: 10.225 | Time: 5.75 seconds
[batch 00019]: Loss 0.898 | Accuracy: 10.016 | Time: 5.78 seconds
[batch 00020]: Loss 0.899 | Accuracy: 9.984 | Time: 5.83 seconds
[batch 00021]: Loss 0.899 | Accuracy: 9.955 | Time: 5.83 seconds
[batch 00022]: Loss 0.

KeyboardInterrupt: ignored